---
title: "Light Under Sea Ice"
author: "Andrew P. Barrett"
date: "2023-02-24"
bibliography: "../docs/references.bib"
format:
    html:
        code-fold: false       
---

```{python}
#| fold-code: true
from IPython.display import Markdown
#from tabulate import tabulate
```

# Light under sea ice

The notebook demonstrates estimating photosynthetically active radiation (PAR) under sea ice following @stroeve_multi-sensor_2021.  The approach of @stroeve_multi-sensor_2021 is based on that of @maykut1971 and @grenfell_optical_1977.  The approach uses apparent optical properties (albedo $\alpha$ and attenuation coefficient $\kappa$) in a two level Bouger-Lambert-Beer exponential decay model.  The first level represents the surface scattering layer - not included for dry snow.  The second level is snow-ice column.

_Need a description of what the snow sea ice column looks like with a diagram and maybe some images_ 

The model is implemented here, first, using basic numpy functions.  It is then extended to account for distributions of ice thickness and snow depth.  

In [23]:
import numpy as np

from beer_lambert_rt.constants import (hssl_ice, hssl_dry_snow, hssl_wet_snow,
                                       k_ice, k_dry_snow, k_wet_snow, k_thin_snow,
                                       i0_ice, i0_dry_snow, i0_wet_snow, i0_melt_ponds,
                                       albedo_open_water)

## Bouger-Lambert-Beer Model

The Bouger-Lambert-Beer model is an approximation of the Radiative Transfer Equation that assumes all attenuation in a medium occurs by absorption.  Single scattering and multiple scattering terms vanish.  Radiative flux $F$ at the base of a column of depth $z$ is given by;

$$
F = F_{0}\exp{(- \kappa z)}
$$ {#eq-expontential_attenuation}

where $F_{0}$ is the radiative flux at the upper surface of the medium of interest and $\kappa$ is the attenuation coefficient of the medium.

Sea ice may be covered by snow.  Furthermore, wet snow and bare sea ice have a surface scattering layer (SSL) that may be up to 10 cm in depth.  Incoming solar radiation is absorbed (or scattered) by this SSL.  @maykut1971 account for this reduction in radiation transmitted through the SSL with a surface transmission function $i_{0}$.  Radiation is then attenuated by the snow (if present) and ice layers below the SSL.  Snow and ice have different attenuation coefficients (_Need Ref_).  The transmission coefficient $T_{ocean}$ for a snow covered ice is the product of two exponential terms, one for snow and one for ice.

$$
T_{ocean} = i_{0,snow} (1 - \alpha_{snow}) \exp(-\kappa_{snow}(h_{snow} - h_{ssl})) \exp(-\kappa_{ice} h_{ice})
$$

where $i_{0,snow}$ is the surface transmission parameter for snow, $\alpha_{snow}$ is the snow albedo, $\kappa_{snow}$ and $\kappa_{ice}$ are attenuation coeffiecnts for snow and ice, and $h_{snow}$, $h_{ssl}$ and $h_{ice}$ are snow depth, thickness of the SSL, and ice thickness respectively.

For bare ice

$$
T_{ocean} = i_{0,ice} (1 - \alpha_{ice}) \exp(-\kappa_{ice} (h_{ice} - h_{ssl}))
$$

Radiative flux at the ice-ocean interface is then

$$
F_{ocean} = F_{0} T_{ocean}
$$


In [28]:
def transmission_snow(hsnow, hice, albedo):
    """Returns transmission coeficient for a snow ice column"""
    return (i0_dry_snow * (1 - albedo) * 
            np.exp(-1 * k_dry_snow * (hsnow - hssl_dry_snow)) * 
            np.exp(-1 * k_ice * hice))

hice = 1.5
hsnow = .3
hpond = 0.
albedo = 0.8
surface_temperature = -10.

transmission_snow(hsnow, hice, albedo)

0.005464744489458511

In [7]:
transmission.calculate_transmission(hice, hsnow, hpond, surface_temperature)

Dry snow covered ice
